In [105]:
import pandas as pd
import glob

In [106]:
class Ball(dict):
    def __init__(self):
        self['score'] = 0
        self['bat'] = 0
        self['wicket'] = None
        self['w_nb'] = True

In [107]:
class Innings(Ball):
    
    def __init__(self):
        self.bat_order = []
        
    def balls(self):
        return [b for b in self.values() if isinstance(b, Ball)]
    
    def runs(self):
        return [b['score'] for b in self.balls()]
    
    def partnerships(self):
        return []
    
    def bat_score(self, batsman):        
        return sum(b['bat'] for b in self.balls() if (b['batsman'] == batsman))

    def bat_balls_faced(self, batsman):        
        return sum(1 for b in self.balls() if (b['batsman'] == batsman) & (b['w_nb']))
    
    def bat_scorecard(self):
        score_board = []
        for batsman in self.bat_order:
            runs = self.bat_score(batsman)
            balls = self.bat_balls_faced(batsman)
            score_board.append((batsman, runs, balls))
        
        sb = pd.DataFrame(data=score_board, columns=['Batsmen', 'Runs', 'Balls Faced'])

        total = ('Total', sum(self.runs()), len(self.balls()))
        extra = ('Extras', total[1] - sb['Runs'].sum(), total[2] - sb['Balls Faced'].sum())
        score_board.append(extra)
        score_board.append(total)

        return pd.DataFrame(data=score_board, columns=['Batsmen', 'Runs', 'Balls Faced'])       
 

In [108]:
class Match(dict):    
    def __init__(self, match_id):
        self.match_id = match_id
        self.team = {}
        
    def bat_score(self, batsman):
        if batsman in self[1].bat_order:
            return self[1].bat_score(batsman)
        elif batsman in self[2].bat_order:
            return self[2].bat_score(batsman)
        else:
            return None
            
        
    def winner(self):
        pass       

In [123]:
class Player(dict):
    
    def __init__(self, name):
        self.name = name
        self.games = []
    
    def __repr__(self):
        return 'Bat Ave : {:0.1f}'.format(self.average())
    
    def total_runs(self):
        pass
    
    def average(self):
        # need to take into account not outs...
        total = [all_games[g].bat_score(self.name) for g in self.games]
        total = [i for i in total if i is not None]
        return sum(total) / len(total)
    

In [124]:
def read_match(match_id):

    game = Match(match_id)
    f = open('IPL/{:s}.yaml'.format(match_id), 'r')

    for line in f.readlines():
        tabs = [a.strip('\n') for a in line.split('  ')]
        len_tabs = len(tabs)
        try:
            var, out = [a.strip() for a in tabs[-1].split(':')]
        except:
            var = None
            out = [a.strip() for a in tabs[-1].split('- ')]

        if len_tabs == 2:
            game[var] = out
            if (var == '- 1st innings'): 
                innings = game[1] = Innings()
            if (var == '- 2nd innings'):
                innings = game[2] = Innings()

        if len_tabs == 5:
            current_ball = innings[var] = Ball()

        if len_tabs == 7:
            if var == 'wicket':
                current_ball['wicket'] = True
            else:
                current_ball[var] = out
            if var == 'batsman':
                if out not in innings.bat_order:
                    innings.bat_order.append(out)       

        if len_tabs == 8:
            if var == 'total':
                current_ball['score'] = int(out)
            if var == 'batsman':
                current_ball['bat'] = int(out)
            if (var == 'wides') | (var == 'no-ball'):
                current_ball['w_nb'] = False
    return game
        
            

In [125]:
all_games = {}
all_batsmen = {}

for file_name in glob.glob('IPL/*.yaml'):
    match_id = file_name.strip('IPL/').strip('.yaml')
    print ('reading in match id {:s}'.format(match_id))
    curr_match = all_games[match_id] = read_match(match_id)
    
    for bat in curr_match[1].bat_order + curr_match[1].bat_order:
        if bat not in all_batsmen.keys():
            all_batsmen[bat] = Player(bat) 
        all_batsmen[bat].games.append(match_id)
        

reading in match id 335982
reading in match id 335983
reading in match id 335984
reading in match id 335985
reading in match id 335986
reading in match id 335987
reading in match id 335988
reading in match id 335989
reading in match id 335990
reading in match id 335991
reading in match id 335992
reading in match id 335993
reading in match id 335994
reading in match id 335995
reading in match id 335996
reading in match id 335997
reading in match id 335998
reading in match id 335999
reading in match id 336000
reading in match id 336001
reading in match id 336002
reading in match id 336003
reading in match id 336004
reading in match id 336005
reading in match id 336006
reading in match id 336007
reading in match id 336008
reading in match id 336009
reading in match id 336010
reading in match id 336011
reading in match id 336012
reading in match id 336013
reading in match id 336014
reading in match id 336015
reading in match id 336016
reading in match id 336017
reading in match id 336018
r

In [126]:
all_batsmen

{'A Ashish Reddy': Bat Ave : 12.0,
 'A Chandila': Bat Ave : 2.0,
 'A Chopra': Bat Ave : 10.0,
 'A Flintoff': Bat Ave : 22.0,
 'A Kumble': Bat Ave : 2.0,
 'A Mishra': Bat Ave : 5.0,
 'A Mithun': Bat Ave : 4.0,
 'A Mukund': Bat Ave : 19.0,
 'A Nehra': Bat Ave : 0.0,
 'A Singh': Bat Ave : 0.0,
 'A Symonds': Bat Ave : 32.0,
 'A Uniyal': Bat Ave : 4.0,
 'AA Bilakhia': Bat Ave : 9.0,
 'AA Chavan': Bat Ave : 4.0,
 'AA Jhunjhunwala': Bat Ave : 20.0,
 'AB Agarkar': Bat Ave : 10.0,
 'AB Dinda': Bat Ave : 1.0,
 'AB McDonald': Bat Ave : 14.0,
 'AB de Villiers': Bat Ave : 37.0,
 'AC Blizzard': Bat Ave : 21.0,
 'AC Gilchrist': Bat Ave : 23.0,
 'AC Thomas': Bat Ave : 0.0,
 'AC Voges': Bat Ave : 27.0,
 'AD Mascarenhas': Bat Ave : 8.0,
 'AD Mathews': Bat Ave : 17.0,
 'AD Nath': Bat Ave : 5.0,
 'AD Russell': Bat Ave : 23.0,
 'AG Murtaza': Bat Ave : 2.0,
 'AG Paunikar': Bat Ave : 8.0,
 'AJ Finch': Bat Ave : 27.0,
 'AL Menaria': Bat Ave : 16.0,
 'AM Nayar': Bat Ave : 15.0,
 'AM Rahane': Bat Ave : 30.0,
 '

In [18]:
all_games['335982'][1].bat_score('BB McCullum')

158

In [7]:
game[1].bat_scorecard()

,Batsmen,Runs,Balls Faced
0,SC Ganguly,10,12
1,BB McCullum,158,73
2,RT Ponting,20,20
3,DJ Hussey,12,12
4,Mohammad Hafeez,5,3
5,Extras,17,4
6,Total,222,124


In [35]:
sb

NameError: name 'sb' is not defined